In [ ]:
# import the required Binary modules: Numpy, Pandas, Seaborn, Matplotlib and other modules
# These modules provide data processing, visualization and machine learning algorithm functions.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm, skew

# Set Palette and Drawing Style
color = sns.color_palette()
sns.set_style('darkgrid')

if __name__ == '__main__':
    # Load dataset: Use the pd.read_csv() function to read data from a CSV file and convert it to DataFrame format.
    poke = pd.read_csv('C:/Users/fortu/Downloads/pokemon0820 (2).csv')
    test_total = poke.base_total.values
    # Split the dataset: Divide the entire dataset into a training set and a test set
    # 75% part of the data is used to train the program as well as the rest of it was used to test the result of machine learning
    train = poke.iloc[:600, :]
    test = poke.iloc[600:801, :]
    # Display Dataset Information: Display number of rows and columns of the dataset
    # check the number of dataset's rows and columns
    print("About this dataset, it has {} rows and {} columns.".format(poke.shape[0], poke.shape[1]))
    # check the type of data
    print("The type of POKE is {}".format(type(poke)))

    # Data Missing in the Dataset
    poke.info()
    # Basic description: including count, mean, standard deviation, minimum value and maximum value
    print(poke.describe().T)
    # Missing values for each attribute in the dashboard: Calculate the percentage of missing values for each attribute in the dataset and save the result in the variable missing_value_poke, including column names and the percentage of missing values.
    percent_missing = poke.isnull().sum() * 100 / len(poke)
    missing_value_poke = pd.DataFrame({
        'column_name': poke.columns,
        'percent_missing': percent_missing
    })
    # The code above is mainly used for loading, processing and analyzing data, including data description statistics, missing value detection and so on.

    # Data preprocessing
    (u, sigma) = norm.fit(poke['base_total'])
    print('\n u = {:.2f} and sigma = {:.2f}\n'.format(u, sigma))
    # # plot the distribution
    # plt.hist(poke.base_total, bins=35)
    # plt.xlabel('base_total')
    # plt.ylabel('Frequency')

    fig = plt.figure()
    res = stats.probplot(poke['base_total'], plot=plt)
    # Data of "base_total" coincides with Normal Distribution. It could be more precise through log transformation.
    poke['base_total'] = np.log1p(poke['base_total'])
    # # check the new distribution
    # get the fitted parameters used by the function
    (u, sigma) = norm.fit(poke['base_total'])
    print('\n u = {:.2f} and sigma = {:.2f}\n'.format(u, sigma))

    # The Data Distribution is more like Normal Distribution through log transformation.

    # Remove all sequences whose data type is "object", and get the index.
    numeric_feats = poke.dtypes[poke.dtypes != "object"].index
    # check the skew of all numerical features
    skewed_feats = poke[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    print("\nSkew in numerical features: \n")
    skewnewss = pd.DataFrame({'Skew': skewed_feats})
    print("There are {} skewed numerical features to Box Cox transform".format(skewnewss.shape[0]))

    from scipy.special import boxcox1p

    skewed_features = skewnewss.index
    lam = 0.15
    for feat in skewed_features:
        poke[feat] = boxcox1p(poke[feat], lam)
    # print(skewnewss.head)

    # Handle blank or missing values
    y_train = train.base_total.values
    y_test = test.base_total.values
    all_data = pd.concat((train, test)).reset_index(drop=True)
    all_data.drop(['base_total'], axis=1, inplace=True)
    all_data.drop(['generation'], axis=1, inplace=True)
    all_data.drop(['is_legendary'], axis=1, inplace=True)
    print("all_data size is {}".format(all_data.shape))
    all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
    all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
    missing_data = pd.DataFrame({'Missing Ratio': all_data_na})

    # Analyse Correlation
    # Select and analyze 10 factors that may affect "base_total"
    column = ['height_m', 'weight_kg', 'base_egg_steps', 'base_happiness', 'hp', 'attack', 'defense', 'sp_attack',
              'sp_defense', 'speed', 'base_total']
    # Imputation Missing Values
    for i in range(len(column) - 1):
        all_data[column[i]] = all_data[column[i]].fillna(0)
    for i in ('name', 'japanese_name', 'pokedex_number', 'type1', 'type2', 'classfication', 'capture_rate', 'abilities',
              'experience_growth', 'percentage_male'):
        all_data.drop([i], axis=1, inplace=True)
    against_column_index = poke.columns.tolist()[1:19]
    for i in against_column_index:
        all_data.drop([i], axis=1, inplace=True)

    plt.figure(figsize=(10, 8))
    for i in range(len(column)):
        plt.subplot(4, 3, i + 1)
        train[column[i]].hist(bins=100, color=color[0])
        plt.xlabel(column[i], fontsize=10)
        plt.ylabel('Frequency', fontsize=10)
    plt.tight_layout() # subscheme generation
    plt.show()

    # BoxPlot
    sns.set_style('ticks')
    sns.set_context("notebook", font_scale=1.1)
    plt.figure(figsize=(10, 8))
    for i in range(len(column)):
        plt.subplot(4, 3, i + 1)
        sns.boxplot(x='base_total', y=column[i], data=train, color=color[0], width=0.6)
        plt.ylabel(column[i], fontsize=10)
    plt.tight_layout()

    # HeatMap
    sns.set_style("dark")
    plt.figure(figsize=(10, 8))
    mcorr = train[column].corr()
    mask = np.zeros_like(mcorr)
    mask[np.triu_indices_from(mask)] = True
    cmap = sns.diverging_palette(240, 20, as_cmap=True)
    g = sns.heatmap(mcorr, mask=mask, cmap=cmap, square=True, annot=True, fmt='0.2f')

    # Fit data by using Liner Regression models
    sns.set_style('ticks')
    sns.set_context("notebook", font_scale=1.4)
    plt.figure(figsize=(6, 4))
    sns.regplot(x='height_m', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='weight_kg', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='attack', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='sp_attack', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='defense', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='sp_defense', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    sns.regplot(x='speed', y='base_total', data=train, scatter_kws={'s': 5}, color=color[7])
    plt.show()

    # Drawing Regression Image
    sns.lmplot(x='base_total', y='defense', hue='sp_attack', data=poke, fit_reg=False, scatter_kws={'s': 10})
    sns.set_style('ticks')
    sns.set_context("notebook", font_scale=1.4)
    plt.figure(figsize=(7, 5))
    cm = plt.cm.get_cmap('RdBu')
    sc = plt.scatter(poke['sp_defense'], poke['base_total'], c=poke['sp_attack'], cmap=cm)
    bar = plt.colorbar(sc)
    bar.set_label('sp_attack', rotation=0)
    plt.xlabel('sp_defense')
    plt.ylabel('base_total')

    # Feature Dimension Reduction
    from sklearn.decomposition import PCA

    pca = PCA(n_components=9)
    all_data_pca = pca.fit_transform(all_data)
    ntrain = train.shape[0]
    ntest = test.shape[0]
    train = all_data[:ntrain]
    test = all_data[ntrain:]

    # Construct Machine Learning models
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, LassoLarsIC
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor
    from sklearn.kernel_ridge import KernelRidge
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import RobustScaler
    from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
    from sklearn.model_selection import KFold, cross_val_score, train_test_split
    from sklearn.metrics import mean_squared_error
    import xgboost as xgb
    import lightgbm as lgb

    # validation function
    n_folds = 5

    def rmsle_cv(model):
        kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
        rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv=kf))
        return rmse

    # different models
    # Ordinary Least Squares
    liner = make_pipeline(RobustScaler(), LinearRegression())
    # Ridge
    ridge = make_pipeline(RobustScaler(), Ridge(alpha=0.9, random_state=1))
    # Lasso
    lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))
    # ElasticNet
    ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=1))
    # Kernel Ridge
    KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
    # calculate the error of each model mean(standard Deviation)
    score = rmsle_cv(liner)
    print("\nLinearRregression score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(ridge)
    print("\nRidge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(lasso)
    print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(ENet)
    print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(KRR)
    print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

    # Ensemble learning(contains Bagging, Boosting and Stacking)
    # Random Forest
    RandomForest = RandomForestRegressor(n_estimators=100, criterion='poisson')
    # The 'criterion' parameter of RandomForestRegressor must be a str among {'friedman_mse', 'poisson', 'squared_error', 'absolute_error'}.
    score = rmsle_cv(RandomForest)
    print("RandomForest score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

    # GBDT
    GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',
                                       min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=5)
    # XGBoost
    model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05, max_depth=3,
                                 min_child_weight=1.7817, n_estimators=2200, reg_alpha=0.4640, reg_lambda=0.8551,
                                 subsample=0.5213, nthread=-1)
    # LightGBM
    model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves=5, learning_rate=0.05, n_estimators=720,
                                  max_bin=55, bagging_freq=5, feature_fraction=0.2319,
                                  feature_fraction_seed=9, bagging_seed=9, min_data_in_leaf=6,
                                  min_sum_hessian_in_leaf=11)

    # calculate the error of these models
    score = rmsle_cv(GBoost)
    print("GBoost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(model_xgb)
    print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    score = rmsle_cv(model_lgb)
    print("LGBMRegressorGBM score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


    # Define Metamodel classes
    class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
        def __init__(self, base_models, meta_model, n_folds=5):
            self.base_models = base_models
            self.meta_model = meta_model
            self.n_folds = n_folds

        # We again fit the data on clones of the original models
        def fit(self, X, y):
            self.base_models_ = [list() for x in self.base_models]
            self.meta_model_ = clone(self.meta_model)
            kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
            # Train cloned base models then create out -of -fold predictions that are needed to train the cloned meta -model
            out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
            for i, model in enumerate(self.base_models):
                for train_index, holdout_index in kfold.split(X, y):
                    instance = clone(model)
                    self.base_models_[i].append(instance)
                    instance.fit(X[train_index], y[train_index])
                    y_pred = instance.predict(X[holdout_index])
                    out_of_fold_predictions[holdout_index, i] = y_pred
            # Now train the cloned meta -model using the out -of -fold predictions as new feature
            self.meta_model_.fit(out_of_fold_predictions, y)
            return self

        # Do the predictions of all base models on the test data and use the averaged predictions as meta -features for the final prediction which is done by the meta -model
        def predict(self, X):
            meta_features = np.column_stack([
                np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
                for base_models in self.base_models_])
            return self.meta_model_.predict(meta_features)


    stacked_averaged_models = StackingAveragedModels(base_models=(ridge, lasso, ENet),
                                                     meta_model=liner)  # 0.0032 (0.0011) 0.002088808594979535
    score = rmsle_cv(stacked_averaged_models)
    print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


    # Loss function
    def rmsle(y, y_pred):
        return np.sqrt(mean_squared_error(y, y_pred))


    # mean_squared_error (MSE) measures the amount of error in statistical models. It assesses the average squared difference between the observed and predicted values. When a model has no error, the MSE equals zero.
    # Fit these data with StackingRegressor, XGBoost and LightGBM models.
    stacked_averaged_models.fit(train.values, y_train)
    stacked_train_predict = stacked_averaged_models.predict(train.values)
    stacked_test_predict = stacked_averaged_models.predict(test)
    stacked_poke_predict = stacked_averaged_models.predict(all_data)
    print("stacked_averaged_models predicted data used for training: {}".format(rmsle(y_train, stacked_train_predict)))

    model_xgb.fit(train, y_train)
    xgb_train_predict = model_xgb.predict(train)
    xgb_test_predict = model_xgb.predict(test)
    xgb_poke_predict = model_xgb.predict(all_data)
    print("model_xgb predicted data used for training: {}".format(rmsle(y_train, xgb_train_predict)))

    GBoost.fit(train, y_train)
    GBoost_train_predict = GBoost.predict(train)
    GBoost_test_predict = GBoost.predict(test)
    GBoost_poke_predict = GBoost.predict(all_data)
    print("GBoost predicted data used for training: {}".format(rmsle(y_train, GBoost_train_predict)))

    KRR.fit(train, y_train)
    KRR_train_predict = KRR.predict(train)
    KRR_test_predict = KRR.predict(test)
    KRR_poke_predict = KRR.predict(all_data)
    print("KRR predicted data used for training: {}".format(rmsle(y_train, KRR_train_predict)))

    model_lgb.fit(train, y_train)
    lgb_train_predict = model_lgb.predict(train)
    lgb_test_predict = model_lgb.predict(test)
    lgb_poke_predict = model_lgb.predict(all_data)
    print("model_lgb predicted data used for training: {}".format(rmsle(y_train, lgb_train_predict)))

    # RMSE on the entire Train data when averaging
    print('RMSLE score on train data')
    print(rmsle(y_train, stacked_train_predict * 0.90 + KRR_train_predict * 0.10 + lgb_train_predict * 0.00))

    # Results
    ensemble = stacked_test_predict * 0.90 + KRR_test_predict * 0.10 + lgb_test_predict * 0.00
    print(rmsle(y_test, ensemble))

    # result = stacked_poke_predict * 0.90 + KRR_poke_predict * 0.10 + lgb_poke_predict * 0.00
    result = stacked_poke_predict
    print(rmsle(test_total, result))

    res_file = pd.read_csv('C:/Users/fortu/Downloads/result.csv')
    res_file['base_total'] = result
    res_file.to_csv('sub.csv')

    # The number of Pokemons in each generation.
    res_file['generation'].value_counts().sort_values(ascending=False).plot.bar()

    # The number of Pokemons in each lineage.
    res_file['type1'].value_counts().sort_values(ascending=True).plot.barh()

    # Distribution of different attributes——ViolinPlot
    base_total = res_file.base_total
    plt.subplots(figsize=(20, 12))
    ax = sns.violinplot(x='type1', y='base_total', data=res_file, palette='muted')

    # Find "Ordinary Pokemon with excellent base_total"
    print(res_file[(res_file.base_total >= 570) & (res_file.is_legendary == 0)].japanese_name.head(10))
    plt.show()